# Bygge AI-agenter med vedvarende hukommelse ved hjælp af Cognee

Denne notebook demonstrerer, hvordan man bygger intelligente AI-agenter med avancerede hukommelsesfunktioner ved hjælp af [**cognee**](https://www.cognee.ai/) - en open source AI-hukommelse, der kombinerer vidensgrafer, semantisk søgning og sessionstyring for at skabe kontekstbevidste AI-systemer.

## 🎯 Læringsmål

Ved slutningen af denne tutorial vil du forstå, hvordan man:
- **Bygger vidensgrafer understøttet af embeddings**: Omformer ustruktureret tekst til struktureret, søgbar viden
- **Implementerer sessionshukommelse**: Skaber samtaler med flere omgange med automatisk kontekstbevaring
- **Bevarer samtaler**: Valgfrit gemmer vigtige interaktioner i langtidshukommelse til fremtidig reference
- **Søger med naturligt sprog**: Tilgår og udnytter historisk kontekst i nye samtaler
- **Visualiserer hukommelse**: Udforsker relationerne i din agents vidensgraf


## 🏗️ Hvad Du Vil Bygge

I denne tutorial vil vi skabe en **Coding Assistant** med vedvarende hukommelse, der:

### 1. **Opbygning af Vidensbase**
   - Indsamler information om udviklerprofiler og ekspertise
   - Behandler principper og bedste praksis inden for Python-programmering
   - Gemmer historiske samtaler mellem udviklere og AI-assistenter

### 2. **Sessionsbevidste Samtaler**
   - Bevarer konteksten på tværs af flere spørgsmål i samme session
   - Gemmer automatisk hvert spørgsmål/svar-par for effektiv hentning
   - Giver sammenhængende og kontekstuelle svar baseret på samtalehistorik

### 3. **Langtidshukommelse**
   - Bevarer vigtige samtaler i en langtidshukommelse
   - Henter relevante minder fra vidensbasen og tidligere sessioner for at informere nye interaktioner
   - Opbygger en voksende vidensbase, der forbedres over tid

### 4. **Intelligent Hukommelseshentning**
   - Bruger grafbevidst semantisk søgning til at finde relevant information på tværs af al gemt viden
   - Filtrerer søgninger efter datasubgrupper (udviklerinfo vs. principper)
   - Kombinerer flere datakilder for at give omfattende svar


## 📋 Forudsætninger & Opsætning

### Systemkrav

Før du starter, skal du sikre dig, at du har:

1. **Python-miljø**
   - Python 3.9 eller nyere
   - Virtuelt miljø (anbefales)
   
2. **Redis Cache** (Påkrævet til sessionhåndtering)
   - Lokal Redis: `docker run -d -p 6379:6379 redis`
   - Eller brug en administreret Redis-tjeneste
   
3. **LLM API-adgang**
   - OpenAI API-nøgle eller andre udbydere (se [dokumentation](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Databasekonfiguration**
   - Ingen konfiguration kræves som standard. Cognee bruger filbaserede databaser (LanceDB og Kuzu)
   - Valgfrit kan du opsætte Azure AI Search som en vektorlagring (se [dokumentation](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Miljøkonfiguration

Opret en `.env`-fil i din projektmappe med følgende variabler:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Forståelse af Cognees hukommelsesarkitektur

### Hvordan Cognee fungerer

Cognee tilbyder et avanceret hukommelsessystem, der går ud over simpel nøgle-værdi-lagring:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Nøglekomponenter:

1. **Knowledge Graph**: Lagrer enheder, relationer og semantiske forbindelser
2. **Vector Embeddings**: Muliggør semantisk søgning på tværs af al lagret information
3. **Session Cache**: Bevarer samtalekontekst inden for og på tværs af sessioner
4. **NodeSets**: Organiserer data i logiske kategorier for målrettet hentning

### Hukommelsestyper i denne vejledning:

- **Persistent Memory**: Langtidslagring i knowledge graph
- **Session Memory**: Midlertidig samtalekontekst i Redis-cache
- **Semantic Memory**: Vektorbaseret lighedssøgning på tværs af al data


## 📦 Installer nødvendige pakker

Installer Cognee med Redis-support til sessionhåndtering:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Initialiser miljø og indlæs biblioteker

Sørg for:
1. At Redis kører (f.eks. via Docker: `docker run -d -p 6379:6379 redis`)
2. At miljøvariabler er sat, før cache-moduler importeres
3. Hvis nødvendigt, genstart kernen og kør cellerne i rækkefølge

Den følgende celle vil:
1. Indlæse miljøvariabler fra `.env`
2. Konfigurere Cognee med dine LLM-indstillinger
3. Aktivere caching til sessionhåndtering
4. Validere, at alle komponenter er korrekt forbundet


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigurer lagringsmapper

Cognee bruger to separate mapper til sine operationer:
- **Data Root**: Gemmer importerede dokumenter og behandlede data
- **System Root**: Indeholder vidensgrafdatabasen og systemmetadata

Vi vil oprette isolerede mapper til denne tutorial som følger:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Nulstil hukommelsestilstand

Før vi begynder at opbygge vores hukommelsessystem, lad os sikre, at vi starter fra bunden.

> 💡 **Tip**: Du kan springe dette trin over, hvis du ønsker at bevare eksisterende minder fra dine tidligere kørseler, når du bruger denne notebook senere.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Del 1: Opbygning af vidensbasen

### Datakilder til vores udviklerassistent

Vi vil indsamle tre typer data for at skabe en omfattende vidensbase:

1. **Udviklerprofil**: Personlig ekspertise og teknisk baggrund  
2. **Python Best Practices**: The Zen of Python med praktiske retningslinjer  
3. **Historiske samtaler**: Tidligere Q&A-sessioner mellem udviklere og AI-assistenter  

Denne mangfoldige data gør det muligt for vores agent at:  
- Forstå brugerens tekniske kontekst  
- Anvende bedste praksis i anbefalinger  
- Lære af tidligere vellykkede interaktioner  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Behandl data til en vidensgraf

Nu vil vi omdanne vores rå tekst til en struktureret hukommelse. Denne proces:

1. **Tilføjer data til NodeSets**: Organiserer information i logiske kategorier
   - `developer_data`: Udviklerprofil og samtaler
   - `principles_data`: Python bedste praksis og retningslinjer

2. **Kører Cognify Pipeline**: Ekstraherer enheder, relationer og skaber embeddings
   - Identificerer nøglekoncepter
   - Skaber semantiske forbindelser mellem relateret information
   - Genererer vektorembeddings

Dette kan tage et øjeblik, mens LLM'en behandler teksten og opbygger grafstrukturen:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualiser vidensgrafen

Lad os udforske strukturen af vores vidensgraf. Visualiseringen viser:
- **Noder**: Enheder udtrukket fra teksten (koncepter, teknologier, personer)
- **Kanter**: Forhold og forbindelser mellem enheder
- **Klynger**: Relaterede koncepter grupperet efter semantisk lighed

Åbn den genererede HTML-fil i din browser for interaktivt at udforske grafen:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Berig hukommelsen med Memify

Funktionen `memify()` analyserer vidensgrafen og genererer intelligente regler om dataene. Denne proces:
- Identificerer mønstre og bedste praksis
- Skaber handlingsorienterede retningslinjer baseret på indholdet
- Etablerer forbindelser mellem forskellige vidensområder

Disse regler hjælper agenten med at træffe mere velinformerede beslutninger, når der besvares spørgsmål. At fange en anden visualisering hjælper dig med at sammenligne, hvordan grafen bliver tættere, når den beriges.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Del 2: Intelligent hukommelsesindhentning

### Demonstration 1: Integration af viden på tværs af dokumenter

Nu hvor vores vidensgraf er opbygget, lad os teste, hvordan Cognee kombinerer information fra flere kilder for at besvare komplekse spørgsmål.

Den første forespørgsel demonstrerer:
- **Semantisk forståelse**: Finder relevante begreber, selv når de ikke er nævnt direkte
- **Krydshenvisning**: Kombinerer udviklerprofil med Python-principper
- **Kontekstuel ræsonnement**: Anvender bedste praksis på specifikke implementeringer

### Demonstration 2: Filtreret søgning med NodeSets

Den anden forespørgsel viser, hvordan man kan målrette specifikke undergrupper af vidensgrafen:
- Bruger `node_name`-parameteren til kun at søge inden for `principles_data`
- Giver fokuserede svar fra et specifikt vidensområde
- Nyttigt, når du har brug for domænespecifik information


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Del 3: Opsætning af sessionsstyring

### Aktivering af samtalehukommelse

Sessionsstyring er afgørende for at bevare konteksten på tværs af flere interaktioner. Her vil vi:

1. **Initialisere brugerens kontekst**: Oprette eller hente en brugerprofil til sporing af sessioner
2. **Konfigurere cache-motor**: Forbinde til Redis for at gemme samtalehistorik
3. **Aktivere sessionsvariabler**: Opsætte kontekstvariabler, der vedvarer på tværs af forespørgsler

> ⚠️ **Vigtigt**: Dette kræver, at Redis kører, og at `CACHING=true` er aktiveret i dit miljø


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Hjælpefunktion: Se sessionshistorik

Denne hjælpefunktion giver os mulighed for at inspicere samtalehistorikken, der er gemt i Redis. Den er nyttig til:
- Fejlfinding af sessionsstyring
- Bekræftelse af, at samtaler bliver gemt i cache
- Forståelse af, hvilken kontekst der er tilgængelig for agenten


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Session 1: Async Support Lab — Første Spørgsmål

Start `async-support-lab` sessionen ved at spørge om telemetri-venlige asyncio-mønstre til en massiv webscraper. Grafen kender allerede til asyncio, aiohttp og overvågningspraksis, så svaret bør afspejle tidligere samtaler, mens det tilpasses det nye spørgsmål.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspicer Session 1 Hukommelse Efter Den Første Udveksling

Ved at køre `show_history(session_1)` umiddelbart efter det indledende spørgsmål bekræftes det, at Cognee skrev både prompten og svaret ind i Redis. Du bør se én post med vejledning om samtidighed.


In [ ]:
await show_history(session_1)

## Session 1: Opfølgning på Datamodeller

Dernæst spørger vi, "Hvornår skal jeg vælge dataclasses frem for Pydantic?" ved at bruge det samme session-id. Cognee bør kombinere Python-principperne med tidligere FastAPI-samtaler for at give nuanceret rådgivning—og demonstrere, at konteksten videreføres inden for en navngivet session.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Bekræft, at session 1-historikken indeholder begge ture

Endnu et `show_history(session_1)`-kald bør vise to spørgsmål og svar. Dette stemmer overens med Mem0-laboratoriets "memory replay"-trin og beviser, at yderligere ture udvider den samme transskription.


In [ ]:
await show_history(session_1)

## Session 2: Design Review Tråd — Ny Session

For at vise isolation mellem tråde starter vi `design-review-session` og beder om logningsvejledning til hændelsesgennemgange. Selvom den underliggende vidensbase er den samme, holder det nye session-id transkriptionerne adskilt.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Gennemgang Session 2 Historie

`show_history(session_2)` bør kun vise design-gennemgangens prompt/svar-par. Sammenlign det med Session 1 for at fremhæve, hvordan Cognee holder uafhængige transkriptioner, mens den genbruger den delte vidensgraf.


In [ ]:
await show_history(session_2)

## Resumé

Tillykke! Du har lige givet din kodeassistent et ægte langtidshukommelseslag drevet af Cognee.

I denne tutorial har du taget rå udviklerindhold (kode, dokumentation, chats) og omdannet det til en graf + vektormemory, som din agent kan søge i, ræsonnere over og løbende forbedre.

Hvad du har lært

1. **Fra rå tekst til AI-hukommelse**: Hvordan Cognee indtager ustrukturerede data og omdanner dem til intelligent, søgbar hukommelse ved hjælp af en kombineret vektor- og vidensgrafarkitektur.

2. **Grafberigelse med memify**: Hvordan man går ud over grundlæggende grafoprettelse og bruger memify til at tilføje afledte fakta og rigere relationer oven på din eksisterende graf.

3. **Flere søgestrategier**: Hvordan man forespørger hukommelsen med forskellige søgetyper (graf-bevidst Q&A, RAG-stil fuldførelse, indsigter, rå tekststykker, kodesøgning osv.) afhængigt af, hvad din agent har brug for.

4. **Visuel udforskning**: Hvordan man inspicerer og fejlretter det, Cognee har bygget, ved hjælp af grafvisualiseringer og Cognee UI, så du faktisk kan se, hvordan viden er struktureret.

5. **Sessionsbevidst hukommelse**: Hvordan man kombinerer kontekst fra en session med vedvarende semantisk hukommelse, så agenter kan huske på tværs af sessioner uden at lække information mellem brugere.


## Vigtige Punkter
1. Hukommelse som en Vidensgraf understøttet af Embeddings

    - **Struktureret forståelse**: Cognee kombinerer en vektor-database og en graf-database, så dine data både kan søges efter betydning og forbindes gennem relationer. Cognee bruger som standard filbaserede databaser (LanceDB til vektor- og Kuzu til graf-database).

    - **Relationsbevidst søgning**: Svar kan baseres ikke kun på "lignende tekst", men også på, hvordan enheder relaterer til hinanden.

    - **Levende hukommelse**: Hukommelseslaget udvikler sig, vokser og forbliver søgbart som én sammenhængende graf.

2. Søge- og Ræsonneringsmodi
    - **Hybrid søgning**: Søgefunktionen kombinerer vektorsimilaritet, grafstruktur og LLM-ræsonnering, fra rå dataopslag til grafbevidst spørgsmål-svar.

    - **Tilpas til opgaven**: Brug completion-stil til naturlige sprog-svar, og chunk-/resumé-/graf-modi, når din agent har brug for rå kontekst eller skal udføre egen ræsonnering.

3. Personlige, Sessionsbevidste Agenter
    - **Sessionskontekst + langtidshukommelse**: Cognee holder kortvarig "tråd"-kontekst adskilt fra langvarig, bruger- eller organisationsniveau hukommelse.

## Anvendelser i den Virkelige Verden

1. **Vertikale AI-Agenter**

    Brug mønsteret fra denne notebook til at drive domænespecifikke copiloter, der bygger på Cognee som deres kerne for søgning og ræsonnering:

- **Udvikler-copiloter**: Kodegennemgang, hændelsesanalyse og arkitekturassistenter, der navigerer gennem kode, API'er, design-dokumenter og tickets som én samlet hukommelsesgraf.

- **Kundeorienterede copiloter**: Support- eller succesagenter, der trækker fra produktdokumentation, FAQ'er, CRM-noter og tidligere tickets med grafbevidst søgning og citerede svar.

- **Interne ekspert-copiloter**: Politik-, juridiske eller sikkerhedsassistenter, der ræsonnerer over sammenhængende regler, retningslinjer og historiske beslutninger i stedet for isolerede PDF'er.

    Cognee er eksplicit positioneret som en vedvarende, præcis hukommelse for AI-agenter og tilbyder en levende vidensgraf, der kan integreres bag din agent og erstatte ad hoc-kombinationer af vektordatabaser og specialtilpasset grafkode.

2. **Forene Datasiloer til Én Hukommelse**

    Den samme tilgang hjælper dig med at bygge et samlet hukommelseslag på tværs af spredte kilder:

- **Fra siloer til én graf**: Indlæs strukturerede (f.eks. databaser) og ustrukturerede data (f.eks. dokumenter, chats) i en enkelt graf understøttet af embeddings, i stedet for separate indekser for hvert system.

- **Tværkilde-ræsonnering med citater**: Udfør flertrinsræsonnering over alt—"forbind" logs, metrics og dokumenter via grafen—og returner stadig velbegrundede svar med kildeangivelse.

- **Videnshubs**: For domæner som bank eller uddannelse bruges Cognee allerede til at forene PDF'er, interne systemer og app-data i én vidensgraf med vektorer, så agenter kan besvare spørgsmål med præcis, citeret kontekst.

## Næste Skridt

Du har implementeret den grundlæggende hukommelsesloop. Her er naturlige udvidelser, du kan prøve på egen hånd (se [Cognee-dokumentation](https://docs.cognee.ai/) for detaljer):

1. **Eksperimentér med tidsmæssig bevidsthed**: Aktivér temporal cognify for at udtrække begivenheder og tidsstempler fra tekst.

2. **Introducér ontologi-drevet ræsonnering**: Definér en OWL-ontologi for dit domæne. Brug Cognee's ontologistøtte, så udtrukne enheder og relationer forankres i det skema, hvilket forbedrer grafkvaliteten og domænespecifikke svar.

3. **Tilføj en feedback-loop**: Lad Cognee justere grafens kantvægte baseret på reel brugerfeedback, så søgningen forbedres over tid i stedet for at forblive statisk.

4. **Tilpas til personalisering og sessionsadfærd**: Brug bruger-ID'er, lejere og datasæt til at give hver person eller team deres eget perspektiv på den delte hukommelsesmotor.

5. **Skalér til mere komplekse agenter**: Integrér Cognee i agentrammer for at bygge multi-agent-systemer, der alle deler det samme hukommelseslag. *Microsoft Agent Framework x Cognee-plugin kommer snart.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal det bemærkes, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
